<a href="https://colab.research.google.com/github/manish2021iitd/Deep-Learning/blob/main/DLassignment3/DLassignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchtext pytorch-lightning wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import wandb
from pytorch_lightning.loggers import WandbLogger

wandb.login(key='e3c892d4f8c9cd9b9043d31938ad090f0a32cec1')

class TransliterationDataset(Dataset):
    def __init__(self, data, char2idx, idx2char, max_len=30):
        self.data = data
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.max_len = max_len
        self.unk_token = '<unk>'
        self.unk_idx = len(char2idx)  # Assign unique index for unknown token

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_word, tgt_word = self.data[idx]

        src_indices = [self.char2idx.get(char, self.unk_idx) for char in src_word]  # Use get() method to handle unknown characters
        tgt_indices = [self.char2idx.get(char, self.unk_idx) for char in tgt_word]

        src_indices = src_indices[:self.max_len]  # Truncate if longer than max_len
        tgt_indices = tgt_indices[:self.max_len]  # Truncate if longer than max_len

        src_indices += [self.char2idx['<pad>']] * (self.max_len - len(src_indices))  # Ensure padding
        tgt_indices += [self.char2idx['<pad>']] * (self.max_len - len(tgt_indices))  # Ensure padding

        return torch.LongTensor(src_indices), torch.LongTensor(tgt_indices)

# Define the Seq2Seq model
class Seq2SeqModel(pl.LightningModule):
    def __init__(self, input_dim, output_dim, embed_dim, hidden_dim, n_layers, cell_type, bidirectional=False, dropout=0.3, learning_rate=0.001):
        super(Seq2SeqModel, self).__init__()
        self.save_hyperparameters()

        if cell_type == "RNN":
            self.rnn = nn.RNN
        elif cell_type == "LSTM":
            self.rnn = nn.LSTM
        elif cell_type == "GRU":
            self.rnn = nn.GRU

        self.encoder = nn.Embedding(input_dim, embed_dim)
        self.encoder_rnn = self.rnn(embed_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)

        self.decoder = nn.Embedding(output_dim, embed_dim)
        self.decoder_rnn = self.rnn(embed_dim, hidden_dim * (2 if bidirectional else 1), n_layers, dropout=dropout, batch_first=True)

        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), output_dim)

        self.learning_rate = learning_rate
        self.bidirectional = bidirectional

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        max_len = tgt.size(1)
        tgt_vocab_size = self.hparams.output_dim

        outputs = torch.zeros(batch_size, max_len, tgt_vocab_size).to(self.device)

        embedded = self.encoder(src)
        encoder_output, hidden = self.encoder_rnn(embedded)

        if self.bidirectional:
            hidden = self._concat_hidden_directions(hidden)

        decoder_input = tgt[:, 0]

        for t in range(1, max_len):
            decoder_input = decoder_input.unsqueeze(1)
            embedded = self.decoder(decoder_input)
            decoder_output, hidden = self.decoder_rnn(embedded, hidden)
            prediction = self.fc(decoder_output.squeeze(1))
            outputs[:, t, :] = prediction

            top1 = prediction.argmax(1)
            decoder_input = tgt[:, t] if np.random.random() < teacher_forcing_ratio else top1

        return outputs

    def _concat_hidden_directions(self, hidden):
        if isinstance(hidden, tuple):  # LSTM hidden state is a tuple (hidden_state, cell_state)
            hidden = (torch.cat((hidden[0][::2], hidden[0][1::2]), dim=2),
                      torch.cat((hidden[1][::2], hidden[1][1::2]), dim=2))
        else:  # GRU and RNN hidden state is a tensor
            hidden = torch.cat((hidden[::2], hidden[1::2]), dim=2)
        return hidden

    def training_step(self, batch, batch_idx):
        src, tgt = batch
        output = self(src, tgt)
        loss = nn.CrossEntropyLoss()(output.view(-1, self.hparams.output_dim), tgt.view(-1))

        preds = output.argmax(dim=2)
        acc = calculate_accuracy(preds, tgt)

        self.log('train_loss', loss, on_step=True, on_epoch=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True)

        return loss

    def validation_step(self, batch, batch_idx):
        src, tgt = batch
        output = self(src, tgt, 0)

        #print(f"Output shape: {output.shape}")  #Debugging
        #print(f"Target shape: {tgt.shape}")  #Debugging

        loss = nn.CrossEntropyLoss()(output.view(-1, self.hparams.output_dim), tgt.view(-1))

        preds = output.argmax(dim=2)
        acc = calculate_accuracy(preds, tgt)

        self.log('val_loss', loss, on_step=True, on_epoch=True)
        self.log('val_acc', acc, on_step=True, on_epoch=True)

        return loss


    def test_step(self, batch, batch_idx):
        src, tgt = batch
        output = self(src, tgt, 0)
        loss = nn.CrossEntropyLoss()(output.view(-1, self.hparams.output_dim), tgt.view(-1))

        preds = output.argmax(dim=2)
        acc = calculate_accuracy(preds, tgt)

        self.log('test_loss', loss, on_step=True, on_epoch=True)
        self.log('test_acc', acc, on_step=True, on_epoch=True)

        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)

# Define a function to calculate accuracy
def calculate_accuracy(predictions, targets, pad_idx=0):
    #print(f"Predictions shape: {predictions.shape}")  # Debug
    #print(f"Targets shape: {targets.shape}")  # Debug

    if len(predictions.shape) == 3:
        _, pred_max = torch.max(predictions, 2)
    elif len(predictions.shape) == 2:
        pred_max = predictions
    else:
        raise ValueError("Unexpected shape for predictions tensor")

    non_pad_elements = targets.ne(pad_idx)
    correct = pred_max.eq(targets).masked_select(non_pad_elements).sum().item()
    total = non_pad_elements.sum().item()
    return correct / total if total > 0 else 0



# Function to load data
def load_data(file_path, max_len=30):
    data = pd.read_csv(file_path, header=None)
    data = [(str(row[0]), str(row[1])) for idx, row in data.iterrows()]
    return data


# Function to create character mappings
def create_char_mappings(data):
    chars = set()
    for src, tgt in data:
        chars.update(list(src))
        chars.update(list(tgt))

    char2idx = {char: idx + 1 for idx, char in enumerate(chars)}
    char2idx['<pad>'] = 0
    idx2char = {idx: char for char, idx in char2idx.items()}

    return char2idx, idx2char


def train_model(train_data_path, val_data_path, test_data_path, char2idx, idx2char, cell_type='LSTM', embed_dim=256, hidden_dim=512, n_layers=2, dropout=0.3, learning_rate=0.001, bidirectional=False, model_class=Seq2SeqModel):
    train_dataset = TransliterationDataset(load_data(train_data_path), char2idx, idx2char)
    val_dataset = TransliterationDataset(load_data(val_data_path), char2idx, idx2char)
    test_dataset = TransliterationDataset(load_data(test_data_path), char2idx, idx2char)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,num_workers=3)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False,num_workers=3)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,num_workers=3)

    model = model_class(
        input_dim=len(char2idx),
        output_dim=len(char2idx),
        embed_dim=embed_dim,
        hidden_dim=hidden_dim,
        n_layers=n_layers,
        cell_type=cell_type,
        bidirectional=bidirectional,
        dropout=dropout,
        learning_rate=learning_rate
    )

    wandb_logger = WandbLogger(project='seq2seq-transliteration')

    trainer = pl.Trainer(
        max_epochs=10,
        devices=1 if torch.cuda.is_available() else None,
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        logger=wandb_logger,
        log_every_n_steps=1
    )

    trainer.fit(model, train_loader, val_loader)
    trainer.test(model, test_loader)


    trainer.fit(model, train_loader, val_loader)
    trainer.test(model, test_loader)


def train_sweep():
    with wandb.init() as run:
        config = wandb.config

        train_data_path = '/kaggle/input/akasharantar/aksharantar_sampled/hin/hin_train.csv'
        val_data_path = '/kaggle/input/akasharantar/aksharantar_sampled/hin/hin_valid.csv'
        test_data_path = '/kaggle/input/akasharantar/aksharantar_sampled/hin/hin_test.csv'

        # Create character mappings
        char2idx, idx2char = create_char_mappings(load_data(train_data_path))

        # Train the model
        train_model(
            train_data_path=train_data_path,
            val_data_path=val_data_path,
            test_data_path=test_data_path,
            char2idx=char2idx,
            idx2char=idx2char,
            cell_type=config.cell_type,
            embed_dim=config.embed_dim,
            hidden_dim=config.hidden_dim,
            n_layers=config.n_layers,
            dropout=config.dropout,
            learning_rate=config.learning_rate,
            bidirectional=config.bidirectional
        )

def run_hyperparameter_sweep():
    sweep_config = {
        'method': 'random',
        'metric': {'name': 'val_loss', 'goal': 'minimize'},
        'parameters': {
            'embed_dim': {'values': [16, 32, 64, 128, 256]},
            'hidden_dim': {'values': [16, 32, 64, 128, 256]},
            'n_layers': {'values': [1, 2, 3]},
            'dropout': {'values': [0.0, 0.1, 0.2, 0.3, 0.4]},
            'learning_rate': {'values': [0.0001, 0.0005, 0.001, 0.005, 0.01]},
            'bidirectional': {'values': [True, False]},
            'cell_type': {'values': ['RNN', 'LSTM', 'GRU']}
        }
    }

    # Initialize the sweep
    sweep_id = wandb.sweep(sweep_config, project='seq2seq-transliteration')

    # Run the sweep
    wandb.agent(sweep_id, train_sweep)

# Run the hyperparameter sweep
run_hyperparameter_sweep()
